In [1]:
!nvidia-smi

Sun Jul  2 09:04:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 102.5 MB/s eta 0:00:

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 14.0 MB/s eta 0:00:00
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


#Imports

In [4]:
import nltk
from nltk.tokenize import sent_tokenize

In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch

In [7]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#**Login to access huggingface model**
Else it throws authentication related error

In [9]:
from huggingface_hub import notebook_login
notebook_login()

Initialize model and tokenizer, same checkpoint is used for both

In [10]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt, use_auth_token=True).to(device)

**download data**

In [11]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-07-02 09:06:58--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-07-02 09:06:59--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.04s   

2023-07-02 09:07:00 (203 MB/s) - ‘summarizer-data.zip’ saved [7903594/

In [12]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [13]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

In [14]:
print(f"Split lengths : {split_lengths}")
print(f"Features : {dataset_samsum['train'].column_names}")
print("\nDilogue :")
print(dataset_samsum["test"][1]["dialogue"])
print("\nSummary : ")
print(dataset_samsum["test"][1]["summary"])

Split lengths : [14732, 819, 818]
Features : ['id', 'dialogue', 'summary']

Dilogue :
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary : 
Eric and Rob are going to watch a stand-up on youtube.


Convert text to features

In [15]:
def convert_text_to_features(sample_text):
  encoded_input = tokenizer(sample_text['dialogue'], max_length = 1024, truncation = True,
                            padding = "max_length",return_tensors = "pt" )

  with tokenizer.as_target_tokenizer():
    encoded_target = tokenizer(sample_text['summary'], max_length = 128, truncation = True,
                            padding = "max_length",return_tensors = "pt" )


  return{
      'input_ids': encoded_input['input_ids'],
      'attention_mask' : encoded_input['attention_mask'],
      'labels' : encoded_target['input_ids']
  }

In [16]:
dataset_samsum_tokenized = dataset_samsum.map(convert_text_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [17]:
dataset_samsum_tokenized['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

**Training**

1. DataCollator is responsible for taking data for training as batches instead of all at once
2. Set training arguments

In [18]:
from transformers.data.data_collator import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model_pegasus)

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="pegasus_samsum",
    warmup_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps = 1e6,
    gradient_accumulation_steps=16,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay = 0.01,
    logging_steps = 10,
    num_train_epochs=1,
)

In [20]:
trainer = Trainer(
    model=model_pegasus,
    args=training_args,
    tokenizer = tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_tokenized['train'],
    eval_dataset=dataset_samsum_tokenized['validation']
)

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.387400,0.340713


Step,Training Loss,Validation Loss
500,0.387400,0.340713


TrainOutput(global_step=920, training_loss=3.7408344235109245, metrics={'train_runtime': 10277.7173, 'train_samples_per_second': 1.433, 'train_steps_per_second': 0.09, 'total_flos': 4.253291617714176e+16, 'train_loss': 3.7408344235109245, 'epoch': 1.0})

**Evaluation**

In [29]:
def generate_batch_size_chunks(list_elements, batch_size):
  """
  split the dataset into smaller batches that we can process simultaneously
  yield successive batch-sized chunks from list of elements
  """

  for i in range(0, len(list_elements), batch_size):
    yield list_elements[i:i+batch_size]


def calculate_metric_on_test_ds(dataset, metric,model, tokenizer,
                                batch_size = 16, device= device,
                                column_text = "article",
                                column_summary = "highlights"):
  article_batches = list(generate_batch_size_chunks(dataset[column_text],batch_size ))
  target_batches = list(generate_batch_size_chunks(dataset[column_summary],batch_size ))

  for article_batches, target_batches in tqdm(
      zip(article_batches, target_batches), total = len(article_batches)):

      inputs = tokenizer(article_batches, max_length = 1024, truncation = True,
                         padding = "max_length", return_tensors = "pt")
      summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                 attention_mask = inputs["attention_mask"].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)

      #Decode the generated texts
      decoded_summaries = [tokenizer.decode(s,skip_special_tokens = True,
                                            clean_up_tokenization_spaces = True)
                  for s in summaries]

      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

      metric.add_batch(predictions = decoded_summaries, references = target_batches)

  #Compute and return the ROUGE scores
  score = metric.compute()
  return score



In [23]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-23-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [30]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'],rouge_metric, trainer.model, tokenizer,
    batch_size = 2,column_text = 'dialogue', column_summary = 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 410/410 [14:40<00:00,  2.15s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018484,0.000284,0.018302,0.018373


In [31]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [32]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [35]:
gen_kwargs = {"length_penalty" : 0.8, "num_beams" :8, "max_length":128}

sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization",model = "pegasus-samsum-model", tokenizer = tokenizer)

print("Dialogue : ")
print(sample_text)

print("Reference Summary : ")
print(reference)

print("Model Summary :")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue : 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Reference Summary : 
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Model Summary :
Amanda can't find Betty's number. Larry called her last time they were at the park together. Hannah would rather she text him. Amanda will text him.
